In [1]:
from functools import partial
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from torch.utils.data import random_split, Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None, train=False):
        
        self.data = pd.read_csv(data_dir)
        try:
            self.data = pd.read_csv(data_dir)
        except CParserError:
            print ("Something wrong the file")
            
        self.labels = self.data.loc[:, 'label']
        self.X = self.data.drop('label', axis=1)
        
        self.transform = transform
        self.transform.fit(self.X)
        temp_train, temp_test, y_train, y_test = train_test_split(self.X, self.labels, test_size=0.2,random_state=0, stratify=self.labels)
        if train:
            self.X = temp_train
            self.labels = y_train
        else:
            self.X = temp_test
            self.labels = y_test
            
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.X.iloc[idx, :].to_numpy().reshape(1, -1)
        label = self.labels.iloc[idx]
        if self.transform:
            features = self.transform.transform(features)
#         if self.target_transform:
#             label = nn.functional.one_hot(torch.tensor(label), num_classes=3)       
        features, label = torch.tensor(features.flatten()).float(), label
        
        return features, label

In [3]:
def load_data(data_dir="/home/rufael.marew/Documents/Academics/AI701/project/World-cup-predictions/data_selected.csv"):

    data_train = CustomDataset(data_dir=data_dir,transform=MinMaxScaler(),train=True,\
                                      target_transform=True)

    data_test = CustomDataset(data_dir=data_dir,transform=MinMaxScaler(),\
                                     target_transform=True)

    return data_train, data_test

In [4]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden):
        super(Net, self).__init__()
        
        self.fcin = nn.Linear(input_dim, hidden[0])
        self.act1 = nn.ReLU()
        
        layer = [self.fcin, self.act1]
        for i in range(len(hidden)-1):
            self.fc = nn.Linear(hidden[i], hidden[i+1])
            self.act = nn.ReLU()
            layer.append(self.fc)
            layer.append(self.act)
       
        self.fcout = nn.Linear(hidden[-1],3)
        layer.append(self.fcout)
        self.soft = nn.Softmax(dim=1)
        layer.append(self.soft)
        
        self.model = nn.Sequential(*layer)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
def gen_net(size,seed=0):
    np.random.seed(seed)
    temp=[]
    for i in range(size):
        temp.append(2**np.random.randint(2, 9))
    return temp
gen_net(6)

[64, 128, 4, 32, 32, 32]

In [7]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    net = Net(28, config["hidden"])
    
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)
    print(f'working on {device} device')
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data()

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    print(len(trainloader))
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

#         with tune.checkpoint_dir(epoch) as checkpoint_dir:
#             path = os.path.join(checkpoint_dir, "checkpoint")
#             torch.save((net.state_dict(), optimizer.state_dict()), path)
        print("accuracy: {}".format(correct/total))
        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [8]:
def test_accuracy(net):
    trainset, testset = load_data()
    
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
#         if torch.cuda.device_count() > 1:
#             net = nn.DataParallel(net)
    net.to(device)
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            features, labels = data
            features, labels = features.to(device), labels.to(device)
            outputs = net(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [14]:
data_dir = os.path.abspath("./database")

num_samples=10
max_num_epochs=10
gpus_per_trial=1
net_size=64

net_arc = [gen_net(net_size) for i in range(net_size)]

config = {
    "hidden": tune.choice(net_arc),            
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

reporter = CLIReporter(
    parameter_columns=["hidden", "lr", "batch_size"],
    metric_columns=["loss", "accuracy", "training_iteration"])

result = tune.run(
    partial(train_cifar, data_dir=data_dir),
    resources_per_trial={"cpu": 32, "gpu": gpus_per_trial},
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter
)

== Status ==
Current time: 2022-11-25 12:12:57 (running for 00:00:00.28)
Memory usage on this node: 37.1/125.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 32.0/32 CPUs, 1.0/1 GPUs, 0.0/52.25 GiB heap, 0.0/26.12 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/rufael.marew/ray_results/train_cifar_2022-11-25_12-12-57
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+----------------------+----------------------+-------------+--------------+
| Trial name              | status   | loc                  | hidden               |          lr |   batch_size |
|-------------------------+----------+----------------------+----------------------+-------------+--------------|
| train_cifar_f8682_00000 | RUNNING  | 10.127.30.24:2859711 | [64, 128, 4, 32_c888 | 0.0510265   |            8 |
| train_cifar_f8682_00001 | PENDING  |                      | [6

== Status ==
Current time: 2022-11-25 12:13:23 (running for 00:00:25.68)
Memory usage on this node: 39.9/125.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 32.0/32 CPUs, 1.0/1 GPUs, 0.0/52.25 GiB heap, 0.0/26.12 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/rufael.marew/ray_results/train_cifar_2022-11-25_12-12-57
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+----------------------+----------------------+-------------+--------------+
| Trial name              | status   | loc                  | hidden               |          lr |   batch_size |
|-------------------------+----------+----------------------+----------------------+-------------+--------------|
| train_cifar_f8682_00000 | RUNNING  | 10.127.30.24:2859711 | [64, 128, 4, 32_c888 | 0.0510265   |            8 |
| train_cifar_f8682_00001 | PENDING  |                      | [6

== Status ==
Current time: 2022-11-25 12:13:43 (running for 00:00:45.74)
Memory usage on this node: 39.9/125.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 32.0/32 CPUs, 1.0/1 GPUs, 0.0/52.25 GiB heap, 0.0/26.12 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/rufael.marew/ray_results/train_cifar_2022-11-25_12-12-57
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+----------------------+----------------------+-------------+--------------+
| Trial name              | status   | loc                  | hidden               |          lr |   batch_size |
|-------------------------+----------+----------------------+----------------------+-------------+--------------|
| train_cifar_f8682_00000 | RUNNING  | 10.127.30.24:2859711 | [64, 128, 4, 32_c888 | 0.0510265   |            8 |
| train_cifar_f8682_00001 | PENDING  |                      | [6

== Status ==
Current time: 2022-11-25 12:14:03 (running for 00:01:05.80)
Memory usage on this node: 39.9/125.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 32.0/32 CPUs, 1.0/1 GPUs, 0.0/52.25 GiB heap, 0.0/26.12 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/rufael.marew/ray_results/train_cifar_2022-11-25_12-12-57
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+----------------------+----------------------+-------------+--------------+
| Trial name              | status   | loc                  | hidden               |          lr |   batch_size |
|-------------------------+----------+----------------------+----------------------+-------------+--------------|
| train_cifar_f8682_00000 | RUNNING  | 10.127.30.24:2859711 | [64, 128, 4, 32_c888 | 0.0510265   |            8 |
| train_cifar_f8682_00001 | PENDING  |                      | [6

Trial name,accuracy,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_f8682_00000,0.442544,2022-11-25_12-23-01,True,,b6856dbe22554a1caf8fa51c42cc69bc,ws-l1-014,10,1.07315,10.127.30.24,2859711,597.734,57.9706,597.734,1669364581,0,,10,f8682_00000,0.00618505
train_cifar_f8682_00001,0.47675,2022-11-25_12-28-03,True,,924728a4c9cd44f7a75da1ff9f249748,ws-l1-014,10,1.0598,10.127.30.24,2881024,296.631,30.1565,296.631,1669364883,0,,10,f8682_00001,0.00637627
train_cifar_f8682_00002,0.447622,2022-11-25_12-28-42,True,,1070463e23bd4ebe81457ad92e2a8a74,ws-l1-014,1,1.07031,10.127.30.24,2894388,33.9322,33.9322,33.9322,1669364922,0,,1,f8682_00002,0.00575542
train_cifar_f8682_00003,0.456708,2022-11-25_12-33-43,True,,b2cef066984642bcaac140bd6386767a,ws-l1-014,10,1.06448,10.127.30.24,2896059,295.423,28.7764,295.423,1669365223,0,,10,f8682_00003,0.00514174
train_cifar_f8682_00004,0.458044,2022-11-25_12-34-21,True,,66b580d4c0e14679b879bbb4d5005ff4,ws-l1-014,1,1.07675,10.127.30.24,2909242,31.4099,31.4099,31.4099,1669365261,0,,1,f8682_00004,0.00456023
train_cifar_f8682_00005,0.443346,2022-11-25_12-38-11,True,,84e8a0d7396442a181fcbb18ad2a3e7d,ws-l1-014,1,1.07112,10.127.30.24,2911145,225.782,225.782,225.782,1669365491,0,,1,f8682_00005,0.00449109
train_cifar_f8682_00006,0.452699,2022-11-25_12-40-19,True,,177c3153410b424c86e483ada0ab3a70,ws-l1-014,2,1.06698,10.127.30.24,2917524,121.927,59.3342,121.927,1669365619,0,,2,f8682_00006,0.00457525
train_cifar_f8682_00007,0.461785,2022-11-25_12-45-15,True,,3a050a435ca94882aad887c012f51de8,ws-l1-014,10,1.06199,10.127.30.24,2922022,289.633,26.6722,289.633,1669365915,0,,10,f8682_00007,0.00511813
train_cifar_f8682_00008,0.451897,2022-11-25_12-49-15,True,,4b7a8b6510ac4125a9dd10a0f0b1d7a9,ws-l1-014,2,1.06692,10.127.30.24,2935322,233.433,115.643,233.433,1669366155,0,,2,f8682_00008,0.00470233
train_cifar_f8682_00009,0.453234,2022-11-25_12-50-20,True,,8f796cd92bf241599b4cdc84b3f17a5e,ws-l1-014,1,1.06899,10.127.30.24,2942683,59.8214,59.8214,59.8214,1669366220,0,,1,f8682_00009,0.00486112


...

== Status ==
Current time: 2022-11-25 12:49:45 (running for 00:36:48.24)
Memory usage on this node: 40.0/125.6 GiB 
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: -1.0632427235444388 | Iter 4.000: -1.0633632969907207 | Iter 2.000: -1.0655817413049884 | Iter 1.000: -1.0679368539753122
Resources requested: 32.0/32 CPUs, 1.0/1 GPUs, 0.0/52.25 GiB heap, 0.0/26.12 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/rufael.marew/ray_results/train_cifar_2022-11-25_12-12-57
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+----------------------+----------------------+-------------+--------------+---------+------------+----------------------+
| Trial name              | status     | loc                  | hidden               |          lr |   batch_size |    loss |   accuracy |   training_iteration |
|-------------------------+------------+----------------------+----------------------+-------------+--------------+---------+--------

== Status ==
Current time: 2022-11-25 12:50:00 (running for 00:37:03.29)
Memory usage on this node: 40.0/125.6 GiB 
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: -1.0632427235444388 | Iter 4.000: -1.0633632969907207 | Iter 2.000: -1.0655817413049884 | Iter 1.000: -1.0679368539753122
Resources requested: 32.0/32 CPUs, 1.0/1 GPUs, 0.0/52.25 GiB heap, 0.0/26.12 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/rufael.marew/ray_results/train_cifar_2022-11-25_12-12-57
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+----------------------+----------------------+-------------+--------------+---------+------------+----------------------+
| Trial name              | status     | loc                  | hidden               |          lr |   batch_size |    loss |   accuracy |   training_iteration |
|-------------------------+------------+----------------------+----------------------+-------------+--------------+---------+--------

== Status ==
Current time: 2022-11-25 12:50:16 (running for 00:37:18.33)
Memory usage on this node: 40.0/125.6 GiB 
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: -1.0632427235444388 | Iter 4.000: -1.0633632969907207 | Iter 2.000: -1.0655817413049884 | Iter 1.000: -1.0679368539753122
Resources requested: 32.0/32 CPUs, 1.0/1 GPUs, 0.0/52.25 GiB heap, 0.0/26.12 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/rufael.marew/ray_results/train_cifar_2022-11-25_12-12-57
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+----------------------+----------------------+-------------+--------------+---------+------------+----------------------+
| Trial name              | status     | loc                  | hidden               |          lr |   batch_size |    loss |   accuracy |   training_iteration |
|-------------------------+------------+----------------------+----------------------+-------------+--------------+---------+--------

2022-11-25 12:50:20,876	INFO tune.py:778 -- Total run time: 2243.21 seconds (2243.04 seconds for the tuning loop).


(func pid=2942683) accuracy: 0.45323356493853556
== Status ==
Current time: 2022-11-25 12:50:20 (running for 00:37:23.06)
Memory usage on this node: 39.8/125.6 GiB 
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 8.000: -1.0632427235444388 | Iter 4.000: -1.0633632969907207 | Iter 2.000: -1.0655817413049884 | Iter 1.000: -1.0684643639331188
Resources requested: 0/32 CPUs, 0/1 GPUs, 0.0/52.25 GiB heap, 0.0/26.12 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/rufael.marew/ray_results/train_cifar_2022-11-25_12-12-57
Number of trials: 10/10 (10 TERMINATED)
+-------------------------+------------+----------------------+----------------------+-------------+--------------+---------+------------+----------------------+
| Trial name              | status     | loc                  | hidden               |          lr |   batch_size |    loss |   accuracy |   training_iteration |
|-------------------------+------------+----------------------+----------------------+------------

In [15]:
best_trial = result.get_best_trial("loss", "min", "last")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
    best_trial.last_result["loss"]))
print("Best trial final validation accuracy: {}".format(
    best_trial.last_result["accuracy"]))

best_trained_model = Net(28, best_trial.config["hidden"])
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
#         if gpus_per_trial > 1:
#             best_trained_model = nn.DataParallel(best_trained_model)
best_trained_model.to(device)

# best_checkpoint_dir = best_trial.checkpoint.value
# model_state, optimizer_state = torch.load(os.path.join(
#     best_checkpoint_dir, "checkpoint"))
# best_trained_model.load_state_dict(model_state)

test_acc = test_accuracy(best_trained_model)
print("Best trial test set accuracy: {}".format(test_acc))

Best trial config: {'hidden': [64, 128, 4, 32, 32, 32, 8, 32, 128, 16, 64, 256, 4, 4, 64, 16, 8, 256, 256, 4, 8, 128, 8, 128, 4, 8, 64, 32, 4, 32, 128, 256, 4, 16, 32, 4, 8, 32, 128, 32, 32, 256, 4, 8, 8, 8, 4, 16, 64, 32, 256, 32, 16, 64, 16, 4, 4, 64, 128, 128, 256, 4, 64, 8], 'lr': 0.050938452790111964, 'batch_size': 16}
Best trial final validation loss: 1.0597962398304899
Best trial final validation accuracy: 0.4767504008551577
Best trial test set accuracy: 0.4583155194527576
